# Proyecto Final - Text Mining

- Autor: Fernando José Sagastume Cerna
- Carnet: 17004989
- Sección: U

El objetivo General de este proyecto es tomar la narración de un partido de soccer, extraer el script y hacer una estimación de las estadisticas del partido.

## Metodología o estrategia

1. **Recolección de datos:** recopilar narraciones detalladas de partidos de fútbol en un formato estructurado, como texto plano o en un formato específico. Además asegurar de tener los datos completos de los eventos claves del partido (goles, tarjetas, etc).
2. **Análisis de Texto y Extracción de Información:** analizar el texto por medio de NLP, identificar patrones en el texto (nombres de equipos, eventos, etc), categorizar y estructurar los eventos.
3. **Creación de la Estructura del Guion:** crear una estructura para representar los eventos en forma de un guió, agrupar eventos por periodo (tiempos) y por tipo (goles, tiros, etc), y asociar eventos con información relevante.
4. **Estimación de Estadísticas del Partido:** utilizar el guión para realizar estimaciones de estadísticas por medio de algoritmos basados en el guión.
5. **Validación y Ajuste:** comparar las estimaciones con las estadísticas reales, y ajustar el alogritmo para mejorar los resultados.
6. **Visualización de Resultados:** presentación de las estadísticas estimadas.